In [1]:
import pandas as pd
import numpy as np
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary 

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("simple_collection.csv")
df.head(3)

,_id,article_content,category,content_type,digest_number,download_status,dt,gather_dt,is_main,keywords,language,model,pk,processed,processed_tags,projects,state,subcategory,title,url
0,6139f2ccc7fae6a4e67e8db8,"Today, Google launched its second cloud region...",NaN,article_tag,NaN,success,2021-07-15T13:09:13Z,2021-07-16T06:17:58.965Z,NaN,NaN,ENGLISH,gatherer.digestrecord,11475,False,[],[1],FILTERED,NaN,Google Launches Second Cloud Region In India,https://analyticsindiamag.com/google-launches-...
1,6139f2ccc7fae6a4e67e8db9,US-based Dun & Bradstreet provides commercial ...,NaN,article_tag,NaN,success,2021-07-15T11:30:00Z,2021-07-16T06:17:58.975Z,NaN,NaN,ENGLISH,gatherer.digestrecord,11476,False,[],[1],FILTERED,NaN,"Interview: Avinash Gupta, Managing Director & ...",https://analyticsindiamag.com/interview-avinas...
2,6139f2ccc7fae6a4e67e8dba,For AI-based software to work well in the real...,NaN,article_tag,NaN,success,2021-07-15T08:30:00Z,2021-07-16T06:17:58.985Z,NaN,NaN,ENGLISH,gatherer.digestrecord,11477,False,[],[1],FILTERED,NaN,How Does Crowdsourcing Data Labelling Work?,https://analyticsindiamag.com/how-does-crowdso...


In [3]:
df = df.drop(columns=['dt', 'gather_dt', '_id', 'url'])
df.head(3)

,article_content,category,content_type,digest_number,download_status,is_main,keywords,language,model,pk,processed,processed_tags,projects,state,subcategory,title
0,"Today, Google launched its second cloud region...",NaN,article_tag,NaN,success,NaN,NaN,ENGLISH,gatherer.digestrecord,11475,False,[],[1],FILTERED,NaN,Google Launches Second Cloud Region In India
1,US-based Dun & Bradstreet provides commercial ...,NaN,article_tag,NaN,success,NaN,NaN,ENGLISH,gatherer.digestrecord,11476,False,[],[1],FILTERED,NaN,"Interview: Avinash Gupta, Managing Director & ..."
2,For AI-based software to work well in the real...,NaN,article_tag,NaN,success,NaN,NaN,ENGLISH,gatherer.digestrecord,11477,False,[],[1],FILTERED,NaN,How Does Crowdsourcing Data Labelling Work?


In [4]:
df.download_status.value_counts()

success    7537
failed      123
Name: download_status, dtype: int64

In [5]:
# удалили незагруженные статьи

df = df.drop(df.loc[df.download_status == 'failed'].index)
df.shape

(7537, 16)

In [6]:
# удалили где нет статей

df = df.drop(df.loc[df.article_content.isna()].index)
df.shape

(7312, 16)

In [7]:
df.language.value_counts()

ENGLISH    6901
RUSSIAN     411
Name: language, dtype: int64

In [8]:
# df_russian = df.loc[df.language == 'RUSSIAN']

In [9]:
# df_russian

In [10]:
df.content_type.value_counts()

article_tag        5361
not_article_tag    1951
Name: content_type, dtype: int64

In [11]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [12]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))
stopword_eng = stopwords.words('english')
print(len(stopword_eng))

morph = pymorphy2.MorphAnalyzer()

151
179


In [14]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopwords = additional_stopwords + stopword_ru + stopword_eng

len(stopwords)

990

In [15]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|�", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form 
                # 0 скоре самый максимальный - остальные нормальные формы первая вторая и тд  реже встречаются
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopwords] # [6]
    
    return words_lem_without_stopwords

In [16]:
df_russian = df.loc[df.language == 'RUSSIAN']

In [17]:
%%time
#Запускаем очистку текста. Будет долго...
df_russian['article_content'] = df_russian['article_content'].apply(lambda x: clean_text(x), 1)

CPU times: user 133 ms, sys: 3.06 ms, total: 136 ms
Wall time: 135 ms


<ipython-input-15-a831bfacbccc>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|�", '', text)
<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
# %%time
# #Запускаем очистку текста. Будет долго...
# df['article_content'] = df['article_content'].apply(lambda x: clean_text(x), 1)

In [19]:
# %%time
# #Запускаем лемматизацию текста. Будет очень долго...
# df['article_content'] = df['article_content'].apply(lambda x: lemmatization(x), 1)

In [20]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
df_russian['article_content'] = df_russian['article_content'].apply(lambda x: lemmatization(x), 1)

CPU times: user 7.5 s, sys: 34.6 ms, total: 7.54 s
Wall time: 7.54 s


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in df_russian['article_content'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts] 
# common_corpus это зеленая матрица - для каждого документа он будет строить что за слова в каждом документе
# текст каждого документа бросаем в doc2bow в наш common_dictionary

In [22]:
from gensim.models import LdaModel

In [23]:
topics_number = 25

In [24]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.

# это не только предопределение нашей модели - это сразу и обучение!!!
lda = LdaModel(common_corpus, num_topics=topics_number, id2word=common_dictionary, passes=10) 
# passes - сколько раз будем сдвигать центроиды

CPU times: user 30.6 s, sys: 7.37 s, total: 38 s
Wall time: 5.3 s


In [25]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [26]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in df_russian['article_content'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['государственный', 'департамент', 'сша', 'предлагать', 'вознаграждение', 'размер', 'миллион', 'информация', 'позволить', 'установить', 'личность', 'киберпреступник', 'атаковать', 'критически', 'важный', 'инфраструктура', 'программвымогатель', 'усилие', 'борьба', 'угроза', 'криптовымогатель', 'координировать', 'целевой', 'белый', 'домомправительство', 'запустить', 'stopransomwaregov', 'предлагать', 'помощь', 'противодействие', 'атака', 'повышение', 'безопасность', 'сеть', 'шаг', 'запуск', 'сеть', 'борьба', 'финансовый', 'преступление', 'министерство', 'финансы', 'структура', 'банк', 'технологический', 'сторона', 'цель', 'эффективный', 'мера', 'борьба', 'отмывание', 'деньга', 'криптовалюта', 'пытаться', 'отслеживать', 'путь', 'средство', 'выплатить', 'качество', 'выкуп', 'злоумышленник', 'фбр', 'возмещение', 'часть', 'выкуп', 'миллион', 'уплатить', 'colonial', 'pipeline', 'маевознаграждение', 'выплачиваться', 'рамка', 'программа', 'государственный', 'департамент', 'вознаграждение', 'пра

[(24, 0.9915011)]

In [27]:
x=lda.show_topics(num_topics=topics_number, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))
# получается это вывели топ 7 слов в каждой тематике по величине вероятности принадлежности к этой теме
# такие вещи как nn надо заносить в стоп слова - снова прогоните - 
# и опять могут мусорные слова быть - это ноормально - это правильно для тематического моделирования
# для тематич моделирования крайне важно сидеть и исключать мусорные токены - млн и тд
# процесс постройки векторов долгий и итеративный

# мы кидаем центроиды и если они будут скатываться к разным минимумам это тоже нормально - разные слова могут быть
# изза стохастичности (кидаем центроиды) то постоянность не будет и скатываться будет в разне места

# можно записать в отдельный список центроиды которые получили и 
# есть методы которые обучаются с нуля но имеют начальное расположение кластеров именно в этих центроидах

topic_0: dect станция трубка базовый containerd контейнер менеджер
topic_1: сборка тред скафандр новость телескоп канал космический
topic_2: конференция язык спо julia стандарт свободный спутник
topic_3: требование безопасность ansible язык оценка оуд разработка
topic_4: млрд apple право россия рынок драйвер китай
topic_5: api linux версия kubernetes сообщество python скрипт
topic_6: dmca вуз звезда студент университет нейтронный amazon
topic_7: резервный квантовый функция поток ibm копия таймер
topic_8: linux astra база мониторинг очень zimbramtablockedextension письмо
topic_9: facebook apple оператор мир оборудование клиент trueconf
topic_10: vpn ipsec настройка доступ безопасность адрес подключение
topic_11: объект материал узел параметр рендёр окно камера
topic_12: git репозиторий annex версия поддержка deckhouse кластер
topic_13: узел kubernetes сеть кластер сборка gitlab тема
topic_14: млрд руб млн google безопасность документ информация
topic_15: программа вредоносный раздел lin